# CT-AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/ctagi.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

  Cloning https://github.com/fuyu-quant/CreateTool-AGI.git to /tmp/pip-req-build-4bu2oz3n
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/CreateTool-AGI.git /tmp/pip-req-build-4bu2oz3n
  Resolved https://github.com/fuyu-quant/CreateTool-AGI.git to commit 5dd12ec111f473d8d54da7ddf32bc3ad7fc94fe0
  Preparing metadata (setup.py) ... done
  Created wheel for createtoolagi: filename=createtoolagi-0.1.3-py3-none-any.whl size=8436 sha256=01bb7248bd300c6e4176052c0f293719babb957106d7abb95f0dc2935b71aec6
  Stored in directory: /tmp/pip-ephem-wheel-cache-hphqcv6s/wheels/7c/ba/46/b1c3a7d693e2bf0b074d226b40ad5cc8f9d360978e0e7fe5d3
Successfully built createtoolagi
  Attempting uninstall: createtoolagi
    Found existing installation: createtoolagi 0.1.2
    Uninstalling createtoolagi-0.1.2:
      Successfully uninstalled createtoolagi-0.1.2


In [2]:
%%capture
!pip install langchain==0.0.167
!pip install openai==0.27.4
!pip install tiktoken==0.4.0
!pip install qdrant-client==1.2.0

In [ ]:
!rm -r ./tools

In [2]:
from createtoolagi import CTAGI

In [3]:
base_model_name = 'gpt-3.5-turbo'
create_model_name = 'gpt-4'
embegging_model_name = 'text-embedding-ada-002'

ctagi = CTAGI(base_model_name='gpt2', create_model_name='gpt2', embegging_model_name='gpt2')

NameError: name 'Creater' is not defined